In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import multiprocessing
from transformers import *
from datasets import load_dataset
from PIL import Image
from torchinfo import summary
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
import time

torch.cuda.empty_cache()
device="cuda" if torch.cuda.is_available() else "cpu"

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.0.1+cu117)
    Python  3.9.18 (you have 3.9.17)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


### Importing Data

In [2]:
def transform(examples):
  inputs = image_processor([img.convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
  }

In [3]:
model_name = "facebook/convnext-base-224-22k"
batch_size = 16
cpu_count=multiprocessing.cpu_count()

image_processor = ConvNextImageProcessor.from_pretrained("facebook/convnext-base-224-22k")

train_ds= load_dataset('./chest_xray/data')
train_ds = train_ds["train"].train_test_split(test_size=0.25) 

labels = train_ds["train"].features["label"].names
dataset = train_ds.with_transform(transform)


train_dataset_loader = torch.utils.data.DataLoader(dataset["train"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = torch.utils.data.DataLoader(dataset["test"], collate_fn=collate_fn, batch_size=batch_size, shuffle=True)


loading configuration file preprocessor_config.json from cache at /home/moose/.cache/huggingface/hub/models--facebook--convnext-base-224-22k/snapshots/a801e9ad0e52947a4583a14696afbe20decbf89e/preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'height', 'width'}, {'shortest_edge'}, {'longest_edge', 'shortest_edge'}, {'longest_edge'}), got 224. Converted to {'shortest_edge': 224}.
Image processor ConvNextImageProcessor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}



Resolving data files:   0%|          | 0/5840 [00:00<?, ?it/s]

### Loading Model

In [4]:
model =  ConvNextForImageClassification.from_pretrained(model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True,
)


model = model.to(device)

loading configuration file config.json from cache at /home/moose/.cache/huggingface/hub/models--facebook--convnext-base-224-22k/snapshots/a801e9ad0e52947a4583a14696afbe20decbf89e/config.json
Model config ConvNextConfig {
  "architectures": [
    "ConvNextForImageClassification"
  ],
  "depths": [
    3,
    3,
    27,
    3
  ],
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_sizes": [
    128,
    256,
    512,
    1024
  ],
  "id2label": {
    "0": "NORMAL",
    "1": "PNEUMONIA"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "NORMAL": "0",
    "PNEUMONIA": "1"
  },
  "layer_norm_eps": 1e-12,
  "layer_scale_init_value": 1e-06,
  "model_type": "convnext",
  "num_channels": 3,
  "num_stages": 4,
  "out_features": [
    "stage4"
  ],
  "out_indices": [
    4
  ],
  "patch_size": 4,
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4"
  ],
  "torch_dtype": "float32",
  "transformers_version": "4.33.3"
}

loading weigh

### Optimizer and accuracy functions

In [5]:
#Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

log_dir = "./convnext_base_224_tensorboard/"
writer = SummaryWriter(log_dir=log_dir)

def accuracy_fn(logits,true):
    return torch.eq(torch.argmax(torch.softmax(logits,dim=1),dim=1).squeeze(),true).sum().item()/len(logits)
    

In [6]:
epochs=10

train_accuracy=[]
test_accuracy=[]
train_loss=[]
test_loss=[]

for i in tqdm(range(epochs)):
    print("Training:")
    model.train()
    
    #Defining accuracy and loss for train and test data
    temp_train_accuracy=[]
    temp_test_accuracy=[]
    temp_train_loss=[]
    temp_test_loss=[]

    net_train_accuracy=0
    net_test_accuracy=0
    net_train_loss=0
    net_test_loss=0
    
    with tqdm(total=len(train_dataset_loader)) as pbar:
        for batch in train_dataset_loader:
                x=batch["pixel_values"].to(device)
                y=batch["labels"].to(device)
    
                #Calculating model output
                result=model(pixel_values=x,labels=y)
                logits=result.logits
    
                #Reseting any old gradient values
                optimizer.zero_grad()
                #loss=loss_fn(logits.squeeze(),y.type(torch.float32))
                loss=result.loss

            
                #Track of metrics        
                accuracy_train=accuracy_fn(logits,y)
                temp_train_accuracy.append(accuracy_train)
                temp_train_loss.append(loss.item())
    
                #Back Propogation
                loss.backward()
            
                #Update Parameters
                optimizer.step()
            
                #Progress Bar Update
                pbar.update(1)
        pbar.close()
    #Tensorboard & Metrics for the dataset
    net_train_accuracy=sum(temp_train_accuracy)/len(temp_train_accuracy)
    net_train_loss=sum(temp_train_loss)/len(temp_train_loss)
    train_accuracy.append(net_train_accuracy)
    train_loss.append(net_train_loss)
    writer.add_scalar("Train Accuracy",net_train_accuracy,i)
    writer.add_scalar("Train Loss",net_train_loss,i)

    #Evaluation
    print("Testing:")
    model.eval()

    with tqdm(total=len(valid_dataset_loader)) as pbar2:
        for batch in valid_dataset_loader:
            x=batch["pixel_values"].to(device)
            y=batch["labels"].to(device)
            
            #Setting inference mode
            with torch.inference_mode():
                result=model(pixel_values=x,labels=y)
                logits=result.logits
                #loss=loss_fn(logits.squeeze(),y.type(torch.float32))
                loss=result.loss

                #Track of metrics
                accuracy_test=accuracy_fn(logits,y)
                temp_test_accuracy.append(accuracy_test)
                temp_test_loss.append(loss.item())

                #Progress Bar Update
                pbar2.update(1)
        pbar2.close()

    #Tensorboard & Metrics for the dataset
    net_test_accuracy=sum(temp_test_accuracy)/len(temp_test_accuracy)
    net_test_loss=sum(temp_test_loss)/len(temp_test_loss)
    test_accuracy.append(net_test_accuracy)
    test_loss.append(net_test_loss)
    writer.add_scalar("Test Accuracy",net_test_accuracy,i)
    writer.add_scalar("Test Loss",net_test_loss,i)

    #Saving the model
    model.save_pretrained(f"./convnext-base-224/checkpoint-{i+1}")
    image_processor.save_pretrained(f"./convnext-base4-224/checkpoint-{i+1}")

    print(f"Epoch {i+1}:\nTrain Accuracy: {net_train_accuracy}  Train Loss: {net_train_loss}  Test Accuracy: {net_test_accuracy}  Test Loss: {net_test_loss}")
    print("\n")

  0%|          | 0/10 [00:00<?, ?it/s]

Training:


  0%|          | 0/274 [00:00<?, ?it/s]

Testing:


  0%|          | 0/92 [00:00<?, ?it/s]

...

  0%|          | 0/274 [00:00<?, ?it/s]

Testing:


  0%|          | 0/92 [00:00<?, ?it/s]

Configuration saved in ./convnext-base-224/checkpoint-10/config.json
Model weights saved in ./convnext-base-224/checkpoint-10/pytorch_model.bin
Image processor saved in ./convnext-base4-224/checkpoint-10/preprocessor_config.json


Epoch 10:
Train Accuracy: 1.0  Train Loss: 0.00019403937994534351  Test Accuracy: 0.9816576086956522  Test Loss: 0.09465228093714635


